In [1]:
from tensorflow.keras.models import load_model
maskNet = load_model("maskDetectorCNN.model")
import numpy as np

In [2]:
import os
data_dir = os.getcwd()+'\\data\\'
data_cat = ["with_mask", "without_mask"]
data_dir

'C:\\Users\\prath\\Desktop\\Capstone\\Face Mask detection\\data\\'

In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

batch_size = 40
target_size=(148, 148)

## Test Data
testing_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset= "validation",
    seed=1,
    image_size= target_size,
    batch_size=batch_size
)

Found 7553 files belonging to 2 classes.
Using 1510 files for validation.


In [4]:

np.round(maskNet.predict(testing_ds))

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [0.]], dtype=float32)

In [5]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

def faceMaskDetector(frame, faceDetectorModel,maskDetectorModel):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (148, 148),(104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    faceDetectorModel.setInput(blob)
    detections = faceDetectorModel.forward()

    faces = []
    locs = []
    preds = []
    
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (148, 148))
            face = img_to_array(face)
            #face = preprocess_input(face)
            
            faces.append(face)
            locs.append((startX, startY, endX, endY))            
            
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskDetectorModel.predict(faces)
        
    return (locs,preds)


def plotSquare(frame, box, color):    
    (startX, startY, endX, endY) = box
    cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

def displayText(displayContent, textX, textY):
    cv2.putText(frame, displayContent, (textX, textY), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

prototxtPath = r"faceDetector\deploy.prototxt"
weightsPath = r"faceDetector\res10_300x300_ssd_iter_140000.caffemodel"

faceDetectorModel = cv2.dnn.readNet(prototxtPath, weightsPath)
maskDetectorModel = load_model("maskDetectorCNN.model")


print("Turning on the camera..")
vs = VideoStream(src=0).start()


while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=700)
    (locs,preds) = faceMaskDetector(frame, faceDetectorModel,maskDetectorModel)
    
    #print(preds)
    
    for (box,pred) in zip(locs,preds):
        if np.round(pred) == 0:
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        plotSquare(frame, box, color)

    
    # Displaying the video as frames
    cv2.imshow("Face-mask Detector", frame)
    # Pressing a wait key to keep the session running
    key = cv2.waitKey(1) 

    # If we press the "x" key from the keyboard, the session ends
    if key == ord("x"):
            break
                
# Preparing to close all windows and quit the video stream
cv2.destroyAllWindows()
vs.stop()

ModuleNotFoundError: No module named 'imutils'